<a href="https://colab.research.google.com/github/13194307/UTS_ML2019_ID13194307/blob/master/ML_A2/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from scipy.special import softmax
from scipy import stats

In [0]:
class NeuralNetwork:
    class Layer:
        class Neuron:
            def __init__(self, inputShape):
                self.weights, self.bias = self.initialiseWeights(inputShape)
                
            def initialiseWeights(self, inputShape):
                weights = np.array([np.random.randn() for _ in range(0, inputShape)]) / math.sqrt(inputShape)
                bias = np.random.randn()
                
                return weights, bias
            
            def getWeightsAndBias(self):
                return self.weights, self.bias
            
            def generateNeuronOutput(self, x):
                return np.dot(self.weights, x) + self.bias
            
            
            
        
        def __init__(self, layerType, neuronsPerLayer, inputShape):
            self.layerType = layerType
            self.numNeurons = neuronsPerLayer
            self.inputShape = inputShape
            self.neurons = [self.Neuron(inputShape) for _ in range(0, neuronsPerLayer)]
            
        def getInputShape(self):
            return self.inputShape
        
        def getNumNeurons(self):
            return self.numNeurons
        
        def generateLayerOutput(self, x):
            layerOutput = []
            
            for neuron in self.neurons:
                neuronOutput = neuron.generateNeuronOutput(x)
                layerOutput.append(neuronOutput)
               
            #print(self.layerType, ":", layerOutput)
            if self.layerType == "Dense":
                #ReLU activation
                layerOutput = np.maximum(layerOutput, 0)
            elif self.layerType == "Output":
                if len(layerOutput) == 1:
                    #Sigmoid activation
                    layerOutput = 1 / (1 + math.exp(-1*layerOutput[0]))
                else:
                    #Softmax activation
                    layerOutput = np.exp(layerOutput)/sum(np.exp(layerOutput))
            else:
                raise NotImplementedError
                    
            return layerOutput
            
        def __str__(self):
            output = ""
            for neuron in self.neurons:
                weights, bias = neuron.getWeightsAndBias()
                output+="\t"
                
                for j in range(0, len(weights)):
                    output+=("w{}: {}, ".format(j, weights[j]))
                    
                output+=("b0: {}\n".format(bias))
            
            return output
        
        
        
        
        
    
    def __init__(self):
        self.layers = []
        self.numLayers = 0
        
    def addLayer(self, layerType, neuronsPerLayer, inputShape=None):
        if inputShape == None:
            inputShape = self.layers[-1].getNumNeurons()
            
        self.layers.append(self.Layer(layerType, neuronsPerLayer, inputShape))
        self.numLayers+=1
        
    def predict(self, x, labels):
        probabilities = []
        
        for sampleX in x:
            prob = self.feedForward(sampleX)
            #print(pred)
            probabilities.append(prob)
      
        predictions = np.argmax(probabilities, axis=1)
        loss = self.calcLoss(probabilities, predictions, labels)
        print(loss)
        return predictions
    
    def feedForward(self, x):
        lastLayerOutput = x
        
        for layer in self.layers:
            lastLayerOutput = layer.generateLayerOutput(lastLayerOutput)
            
        return lastLayerOutput
    
    def calcLoss(self, prob, pred, actual):
        #print(np.log([prob[i][actual[i]] for i in range(0, len(actual))]))
        loss = -1*sum(np.log([prob[i][actual[i]] for i in range(0, len(actual))]))
        return loss
        
    def __str__(self):
        output = ""
        
        for i in range(0, self.numLayers):
            output+=("Layer {}:\n".format(i+1))
            output+=str(self.layers[i])
        
        return output

In [0]:
nn = NeuralNetwork()
nn.addLayer("Dense", 20, inputShape=4)
nn.addLayer("Dense", 20)
nn.addLayer("Dense", 10)
nn.addLayer("Output", 3)
#print(nn)

In [0]:
from sklearn.datasets import load_iris

iris_X, iris_y = load_iris(True)

In [0]:
from sklearn.preprocessing import MinMaxScaler

iris_X_trimmed = iris_X

scaler = MinMaxScaler()
iris_X_scaled = scaler.fit_transform(iris_X_trimmed)
iris_X_zscore = stats.zscore(iris_X_trimmed)

In [121]:
pred = nn.predict(iris_X_zscore, iris_y)
accuracy_score(pred, iris_y)

[-2.77152267 -2.65329343 -2.84443032 -2.76971499 -2.8038399  -2.54258437
 -2.91226092 -2.84878741 -2.66364486 -2.79693923 -2.6113142  -2.94354231
 -2.69917471 -2.83629605 -2.4505205  -2.41749829 -2.52438895 -2.73088899
 -2.49997512 -2.65105134 -2.73327674 -2.66650888 -2.83024239 -2.75023836
 -3.0167926  -2.6676129  -2.7817369  -2.74789921 -2.74857567 -2.86818063
 -2.76973781 -2.60761118 -2.5583149  -2.46610846 -2.75527276 -2.76535435
 -2.55271394 -2.889718   -2.72411104 -2.81306989 -2.74723513 -2.56639425
 -2.89180421 -2.65078346 -2.72465433 -2.61683318 -2.71884099 -2.8644132
 -2.66073541 -2.84274402 -0.18981897 -0.16725067 -0.16013614 -0.12322669
 -0.12845753 -0.1279022  -0.15720041 -0.13718861 -0.14275477 -0.15379191
 -0.10835147 -0.14750906 -0.11328437 -0.1194589  -0.16301983 -0.17892798
 -0.13813931 -0.13272604 -0.11094802 -0.13703133 -0.13768022 -0.14133673
 -0.11029977 -0.11249504 -0.14673934 -0.16114819 -0.13182344 -0.13533194
 -0.13453851 -0.15150061 -0.13162128 -0.13236742 -0.

0.3333333333333333

In [115]:
print(pred)

[2 0 0 0 2 2 0 0 0 0 2 0 0 0 2 2 2 0 2 2 0 2 2 0 0 0 0 2 0 0 0 0 2 2 0 0 2
 2 0 0 0 1 0 0 2 0 2 0 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 2 2 2 2 2 2 2 2 2 2 1
 2 2 2 2 2 1 0 2 2 2 2 2 2 2 1 1 1 2 1 2 1 2 1 1 2 2 1 1 2 2 2 2 2 2 2 2 2
 2 1]
